# Query ConsDB

A basic notebook to learn about querying the ConsDb

Craig Lage - 17-Apr-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
import pandas as pd
import os
os.environ["no_proxy"] += ",.consdb"
from lsst.summit.utils import ConsDbClient
from lsst.summit.utils.utils import computeCcdExposureId

In [ ]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
print(client.schema())  # list the instruments

In [ ]:
client.schema('lsstcam') # List tables for an instrument

In [ ]:
client.schema('lsstcam', 'exposure_quicklook') # List values in a table

In [ ]:
# Get a particular value from a particular visit
query = 'SELECT * from cdb_lsstcam.exposure_quicklook '
query += 'where day_obs = 20250506'
data = client.query(query)
print(len(data))
data

In [ ]:
# Get all of the values for a particular table and visit
data = client.query('SELECT * from cdb_latiss.exposure_quicklook')
for name in data.colnames:
    print(name, data[name].value[0])

In [ ]:
data = client.query('SELECT * from cdb_lsstcam.exposure where exposure_id = 2025042700545')
for name in data.colnames:
    print(name, data[name].value[0])


In [ ]:
ccdvisit_id = computeCcdExposureId('latiss', 2024053000592, 0)
print(ccdvisit_id)
data = client.query(f'SELECT * from cdb_latiss.ccdvisit1_quicklook where ccdvisit_id = {ccdvisit_id}')
for name in data.colnames:
    print(name, data[name].value[0])

In [ ]:
# I guess this just isn't populated yet?
data = client.query('SELECT ccdvisit_id from cdb_latiss.ccdvisit1_quicklook where ccdvisit_id > 1')
data

In [ ]:
# I guess this just isn't populated yet?
data = client.query('SELECT visit_id from cdb_latiss.visit1_quicklook where visit_id > 1')
data

In [ ]:
data = client.query('SELECT obs_start from cdb_lsstcomcamsim.exposure where day_obs = 20240621 and seq_num = 2')
data

In [ ]:
data = client.query('SELECT seq_num, obs_start, azimuth, zenith_distance from cdb_lsstcomcamsim.exposure where (seq_num > 0 and seq_num < 10 and day_obs = 20240621)')
data

In [ ]:
data = client.query('SELECT * from cdb_lsstcomcamsim.exposure where exposure_id = 7024062700764')
for name in data.colnames:
    print(name, data[name].value[0])
